<a href="https://colab.research.google.com/github/HammadN98/inadimplencia_em_cartoes_de_credito/blob/main/inadimplecia_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd